### THIS FILE CONTAINS CONFIDENTIAL DATA, ONLY RAW CODE HAS BEEN UPLOADED. INDIVIDUAL CELL OUTPUTS HAVE BEEN OMITTED. ANONYMIZATION KEY AND RAW VIEWERSHIP DATA WILL NOT BE LOADED TO DATABASE AND UNAVAILABLE. THIS CODE WILL DOCUMENT THE CLEANING FOR THE VIEWERSHIP DATA

Merging multiple CSVs to one

In [ ]:
### Data Processing Into DataFrames
# Import dependancies
import pandas as pd
from datetime import datetime, timedelta, date
from pathlib import Path
import pycountry_convert as pc

In [ ]:
# Merging Minute level aggregation into one table

#Determine input date range
start_date = "2021-02-18"
end_date = "2022-11-06"

#string to datetime
start_date_dt = datetime.strptime(start_date, '%Y-%m-%d').date()
end_date_dt = datetime.strptime(end_date, '%Y-%m-%d').date()

In [ ]:
# define a function that will read all the csv files and combine them into one dataframe.

def minute_data_aggregation_condenser(start, end):
    current_date = start
    combined_df = pd.DataFrame(pd.read_csv(f"Resources/minutelevelsessionaggregations-qwest-{start_date_dt}.csv"))
    counter = 1

    while current_date <= end:
        try:
            current_date = current_date + timedelta(days=1)
            current_data = pd.read_csv(f"Resources/minutelevelsessionaggregations-qwest-{current_date}.csv")
            current_df = pd.DataFrame(current_data)
            combined_df = pd.concat([combined_df, current_df])
            counter += 1
        except:
            current_date = current_date + timedelta(days=1)
    else:
        print(f"Data Merge Complete, {counter} files have been merged into a dataframe and exported as merged_aggregate_data{date.today()}.csv")
        return combined_df

combined_df = minute_data_aggregation_condenser(start_date_dt, end_date_dt)
combined_df.head()

In [ ]:
#check df length
print(len(combined_df))

In [ ]:
# Check columns in df
combined_df.columns

In [ ]:
#Extra column named channel;time;content_id;country;total_sessions;total_session_duration_seconds indicates error
# Check that column
len(combined_df.columns)
combined_df['channel;time;content_id;country;total_sessions;total_session_duration_seconds'].value_counts()

In [ ]:
# remove all rows containing bad data
combined_df = combined_df[pd.isnull(combined_df['channel;time;content_id;country;total_sessions;total_session_duration_seconds'])]

# Column was semicolon separated rather than comma separated on 2022-06-26
semicolon_data = pd.read_csv("Resources/minutelevelsessionaggregations-qwest-2022-06-26.csv", sep=';')
semicolon_df = pd.DataFrame(semicolon_data)
semicolon_df.head()

#combine with complete dataframe
combined_df=pd.concat([combined_df, semicolon_df])

#remove 'channel;time;content_id;country;total_sessions;total_session_duration_seconds' column
combined_df = combined_df.drop(columns=['channel;time;content_id;country;total_sessions;total_session_duration_seconds'])
combined_df.head()

In [ ]:
#check length of df to make sure it matches with previous 
print(len(combined_df))

Anonymizing Key and Channel Data

In [ ]:
# import anonymization key
anon_key = pd.read_csv("Resources/Anonymization Keys.csv")
anon_key_df = pd.DataFrame(anon_key)
anon_key_df

anon_key_op_df = anon_key_df[['Operator', 'anonymization key']]
anon_key_chan_df = anon_key_df[['Channel', 'Anonymization key (Genre)']]
anon_key_chan_df = anon_key_chan_df.dropna()

In [ ]:
#match channel with operator ### EXPECTED TIME 349minutes REFACTOR THIS CODE IN THE FUTURE TO RUN LIKE THE CONTENT_ID
def string_parser_OPS (string):
    for ops in anon_key_op_df['Operator']:
        if string.str.contains(ops.lower()).any():
            return anon_key_op_df.loc[anon_key_op_df['Operator']== ops, 'anonymization key'].item()

combined_df["Operator"] = combined_df[['channel']].apply(string_parser_OPS, axis =1, result_type='expand')

In [ ]:
#match channel with channel ### EXPECTED TIME 244minutes REFACTOR THIS CODE IN THE FUTURE TO RUN LIKE THE CONTENT_ID
def string_parser_CHAN (string):
    for ops in anon_key_chan_df['Channel']:
        if string.str.contains(ops.lower()).any():
            return anon_key_chan_df.loc[anon_key_chan_df['Channel']== ops, 'Anonymization key (Genre)'].item()

combined_df["Channel"] = combined_df[['channel']].apply(string_parser_CHAN, axis =1, result_type='expand')

Anonymizing Program Code and obtaining Genre 

In [ ]:
content_data = pd.read_csv('Resources/Media Library.csv')
content_df = pd.DataFrame(content_data)

# Add PRO_ prefix to ID to get Program ID
content_df['PRO_CONTENT_ID'] = 'PRO_' + content_df['🎦  ID']
#invert rows sort by largets to smallest so that PRO_3000 is found as PRO_3000 instead of PRO_3
content_df = content_df.sort_index(ascending=False)
content_df.head()

In [ ]:
playlist_data = pd.read_csv('Resources/Playlist With Music Tags.csv')
playlist_df = pd.DataFrame(playlist_data)
# drop to just name and genre
playlist_df = playlist_df[['Name','🎯  TAG Music Styles']]
playlist_df.head()

In [ ]:
#generate unique playlist IDs and Anonymize

#separate string and number from title
unique_playlists = playlist_df['Name'].str.extract(r'([a-z]*)([0-9]*)')

# find unique values in column containing text and make an index number to make a key
unique_playlist_letters = pd.DataFrame(unique_playlists[0].unique())
unique_playlist_letters['index_key'] = unique_playlist_letters.index

# merge on letters
keyed_text_playlist = (unique_playlists.merge(unique_playlist_letters, left_on=0, right_on=0))

# Create anonymization key
playlist_df['anonymized_key'] = ('PLY_') + (keyed_text_playlist['index_key'].astype(str)) + (keyed_text_playlist[1].astype(str))
playlist_df.head()

In [ ]:
# filter out content_id with regex to get program number
regex_list = [r'(PRO_*\d*)_[A-Z]',r'(PRO_*\d*\w*)', r'pro(\d{1,4})', r'pro_(\d*\w*)',r'(^\d{1,4})[a-z]',r'pr\d*[a-z]*(\d*)']
regex_filtered_content_id = combined_df.content_id.str.extract('|'.join(regex_list))
# add PRO_ prefix to extracted numbers
for i in range(len(regex_list)-1, 1, -1):
        regex_filtered_content_id[i]='PRO_' + regex_filtered_content_id[i]
#Merge all columns
for i in range(len(regex_list), 0, -1):
    if i-2 >= 0:
        regex_filtered_content_id[i-2] = regex_filtered_content_id[i-2].fillna(regex_filtered_content_id[i-1])
regex_filtered_content_id = regex_filtered_content_id[[0]]

In [ ]:
# add column to the dataframe
combined_df['filtered_content_id'] = regex_filtered_content_id

In [ ]:
# merge (VLOOKUP) playlists on exact title but dont drop rows
combined_df = combined_df.merge(playlist_df, how='left', left_on='content_id', right_on='Name')
combined_df = combined_df.drop(['Name'], axis=1)

#merge the filtered id for programs with anonymized id for playlists then drop anonymized key column
combined_df['filtered_content_id'] = combined_df['filtered_content_id'].fillna(combined_df['anonymized_key'])
combined_df=combined_df.drop(['anonymized_key'], axis=1)
combined_df

In [ ]:
# check the content_id of rows that have nan for filtered_content_id
missed_content_ids = combined_df[combined_df['filtered_content_id'].isnull()]
missed_content_ids = missed_content_ids[['content_id']]
missing_content_id_df = pd.DataFrame(missed_content_ids['content_id'].unique())
missing_content_id_df.head()

In [ ]:
# merge (Vlookup) programs with viewership data
combined_genre_df = (combined_df.merge(content_df, left_on='filtered_content_id', right_on='PRO_CONTENT_ID'))

# merge tag columns containing genre
combined_genre_df['🎯  TAG Music Styles'] = combined_genre_df['🎯  TAG Music Styles'].fillna(combined_genre_df['🎯  TAG Music Styles (from 🎥 Films)'])

# drop rows where genre is nan
combined_genre_df = combined_genre_df[combined_genre_df['🎯  TAG Music Styles'].notna()]

combined_genre_df.head()
# 80.38% Match Rate

In [ ]:
#Trim useless columns out, and fix column names
trimmed_clean_merged_minute_aggregation = combined_genre_df[['time', 'country', 'total_sessions', 'total_session_duration_seconds', 'Operator', 'Channel', 'filtered_content_id', '🎯  TAG Music Styles (from 🎥 Films)']]
trimmed_clean_merged_minute_aggregation = trimmed_clean_merged_minute_aggregation.rename({'time':'time', 'country':'country', 'total_sessions':'total_sessions', 'total_session_duration_seconds':'total_session_duration_seconds', 'Operator':'operator', 'Channel':'channel','filtered_content_id':'cleaned_content_id', '🎯  TAG Music Styles (from 🎥 Films)':'genre'}, axis='columns')
trimmed_clean_merged_minute_aggregation.head()

In [ ]:
# convert time to datetime
trimmed_clean_merged_minute_aggregation['time']=pd.to_datetime(trimmed_clean_merged_minute_aggregation['time'])
trimmed_clean_merged_minute_aggregation

In [ ]:
# check data types
trimmed_clean_merged_minute_aggregation.info()

Adding Region Code By Bucketing Countries Into Regions

In [ ]:
# add region code by bucketting into regions

# Bucketing countries into regions
# Conversion Function
def convert(row):
    try:
        country_code = pc.country_name_to_country_alpha2(row.country, cn_name_format = "default")
        continent_code = pc.country_alpha2_to_continent_code(country_code)
        return continent_code
    except:
        print(f'{row.country} not found')

# Changing the Country Name so it doesnt interfere with the function
trimmed_clean_merged_minute_aggregation['country'] = trimmed_clean_merged_minute_aggregation['country'].replace("Kosovo", "Albania")
trimmed_clean_merged_minute_aggregation['country'] = trimmed_clean_merged_minute_aggregation['country'].replace("Runion", "Réunion")
trimmed_clean_merged_minute_aggregation['country'] = trimmed_clean_merged_minute_aggregation['country'].replace("U.S. Virgin Islands", "Others")
trimmed_clean_merged_minute_aggregation['country'] = trimmed_clean_merged_minute_aggregation['country'].replace("Western Sahara", "Morocco")
trimmed_clean_merged_minute_aggregation['country'] = trimmed_clean_merged_minute_aggregation['country'].replace("Vatican City", "Italy")
trimmed_clean_merged_minute_aggregation['country'] = trimmed_clean_merged_minute_aggregation['country'].replace("St Vincent and Grenadines", "Saint Vincent and the Grenadines")
trimmed_clean_merged_minute_aggregation['country'] = trimmed_clean_merged_minute_aggregation['country'].replace("Timor-Leste", "Others")
trimmed_clean_merged_minute_aggregation['country'] = trimmed_clean_merged_minute_aggregation['country'].replace("St Kitts and Nevis", "the Federation of Saint Christopher and Nevis")
trimmed_clean_merged_minute_aggregation['country'] = trimmed_clean_merged_minute_aggregation['country'].replace("So Tom and Prncipe", "Others")
trimmed_clean_merged_minute_aggregation['country'] = trimmed_clean_merged_minute_aggregation['country'].replace("Sint Maarten", "Saint Martin")
trimmed_clean_merged_minute_aggregation['country'] = trimmed_clean_merged_minute_aggregation['country'].replace("Saint Barthlemy", "Others")
trimmed_clean_merged_minute_aggregation['country'] = trimmed_clean_merged_minute_aggregation['country'].replace("Bonaire Sint Eustatius and Saba", "Others")
trimmed_clean_merged_minute_aggregation['country'] = trimmed_clean_merged_minute_aggregation['country'].replace("DR Congo", "Democratic Republic of the Congo")
trimmed_clean_merged_minute_aggregation['country'] = trimmed_clean_merged_minute_aggregation['country'].replace("Curaao", "Curaçao")
trimmed_clean_merged_minute_aggregation['country'] = trimmed_clean_merged_minute_aggregation['country'].replace("Åland", "Finland")

# Create a [region] column based on country names with pycountry
trimmed_clean_merged_minute_aggregation['region'] = trimmed_clean_merged_minute_aggregation.apply(convert, axis=1)

#getting Unique Regions
trimmed_clean_merged_minute_aggregation['region'].unique()

In [ ]:
# mapping continent codes to continent names 
continent_names = { 'NA' : 'North America',
                    'AS' : 'Asia',
                    'EU' : 'Europe',
                    'SA' : 'South America',
                    'AF' : 'Africa',
                    'OC' : 'Oceania'}

trimmed_clean_merged_minute_aggregation['region'] = trimmed_clean_merged_minute_aggregation['region'].map(continent_names)
trimmed_clean_merged_minute_aggregation

Exporting final cleaned CSV and missing content_ids for further data cleaning if necesary

In [ ]:
# Output new dataframe to CSV
filepath = Path(f'Resources/cleaned_merged_minute_aggregate_data-{date.today()}.csv')
trimmed_clean_merged_minute_aggregation.to_csv(filepath)

# Output problem content_id dataframe to CSV
filepath = Path(f'Resources/problematic_content_id.csv')
missing_content_id_df.to_csv(filepath)